In [2]:
import time
import openai
import pandas as pd
import glob
from tqdm import tqdm
import os
import dotenv
import requests
import json
import csv

In [3]:
# Read the input file into a dataframe
df = pd.read_csv("test_data.csv")

# Get the 'text' column from the dataframe
half_rows = len(df['LyricsClean']) // 2
text_column = df['LyricsClean'][:half_rows]

# Convert the 'text' column to a list
text_list = text_column.tolist()

# Create the desired style
lyrics = text_list

In [4]:
def dividir_por_tokens(lista):
    grupos = []
    grupo_atual = []
    contador_tokens = 0

    for string in lista:
        tokens = len(string)

        if contador_tokens + tokens <= 1000:
            grupo_atual.append(string)
            contador_tokens += tokens
        else:
            grupos.append(grupo_atual)
            grupo_atual = [string]
            contador_tokens = tokens

    if grupo_atual: 
        grupos.append(grupo_atual)
    
    return grupos

In [11]:
dotenv.load_dotenv()


def analise_sentimento(texto: str) -> str:
    CHAVE_API= os.getenv("CHAVE_API", None)
    modelo_engine = "text-davinci-003"
    #FIRST_COMMANDO TO ZERO-SHOT
    #comando = f"Analyze the following lyrics and determine if the sentiment is: POSITIVE, NEGATIVE or NEUTRAL. Return only a single word, either POSITIVE, NEGATIVE or NEUTRAL by lyrics: '{texto}'"
    
    #SECOND_COMMANDO TO ONE-SHOT
    #comando = f"The following lyrics is POSITIVE: ""payphone trying call home change spent times gone baby wrong plans made two hard remember people used even harder picture next say late make late try time wasted bridges burned wasted nights turned lights paralyzed still stuck time called love even sun sets paradise payphone trying call home change spent times gone baby wrong plans made two happy ever exist would still holding like fairy tales full one stupid love song sick turned back tomorrow cause forgot yesterday gave love borrow gave away expect fine expect care said bridges burned wasted nights turned lights paralyzed still stuck time called love even sun sets paradise payphone trying call home change spent times gone baby wrong plans made two happy ever exist would still holding like fairy tales full one stupid love song sick payphone baby hang tell need baby begging please go tell need payphone trying call home change spent times gone baby wrong plans made two happy ever exist would still holding like fairy tales full one stupid love song sick payphone"". The following lyrics is NEGATIVE: ""i threw a wish in the well dont ask me ill never tell i looked to you as it fell and now youre in my way i trade my soul for a wish pennies and dimes for a kiss i wasnt looking for this but now youre in my way your stare was holdin ripped jeans skin was showin hot night wind was blowin where you think youre going baby hey i just met you and this is crazy but heres my number so call me maybe its hard to look right at you baby but heres my number so call me maybe hey i just met you and this is crazy but heres my number so call me maybe and all the other boys try to chase me but heres my number so call me maybe you took your time with the call i took no time with the fall you gave me nothing at all but still youre in my way i beg and borrow and steal at first sight and its real i didnt know i would feel it but its in my way your stare was holdin ripped jeans skin was showin hot night wind was blowin where you think youre going baby hey i just met you and this is crazy but heres my number so call me maybe its hard to look right at you baby but heres my number so call me maybe hey i just met you and this is crazy but heres my number so call me maybe and all the other boys try to chase me but heres my number so call me maybe before you came into my life i missed you so bad i missed you so bad i missed you so so bad before you came into my life i missed you so bad and you should know that i missed you so so bad  its hard to look right at you baby but heres my number so call me maybe hey i just met you and this is crazy but heres my number so call me maybe and all the other boys try to chase me but heres my number so call me maybe before you came into my life i missed you so bad i missed you so bad i missed you so so bad before you came into my life i missed you so bad and you should know that so call me maybe"". The following lyrics is NEUTRAL: ""oh its just me myself and i solo ride until i die cause i got me for life oh i dont need a hand to hold even when the night is cold i got that fire in my soul and as far as i can see i just need privacy plus a whole lot of tree fuck all this modesty i just need space to do me get a world that theyre tryna see a stella maxwell right beside of me a ferrari im buyin three a closet of saint laurent get what i want when i want cause this hunger is driving me yeah i just need to be alone i just need to be at home understand what im speaking on if time is money i need a loan but regardless ill always keep keepin on fuck fake friends we dont take ls we just make ms while yall follow we just make trends im right back to work when that break ends oh its just me myself and i solo ride until i die cause i got me for life  oh i dont need a hand to hold even when the night is cold i got that fire in my soul i dont need anything to get me through the night except the beat thats in my heart yeah its keeping me alive  i dont need anything to make me satisfied  cause the music fills me good and it gets me every time yeah and i dont like talking to strangers so get the fuck off me im anxious im tryna be cool but i may just go ape shit say fuck yall to all of yall faces it changes though now that im famous everyone knows how this lifestyle is dangerous but i love it the rush is amazing celebrate nightly and everyone rages i found how to cope with my anger im swimming in money swimming in liquor my liver is muddy but its all good im still sippin this bubbly this shit is lovely this shit aint random i didnt get lucky made it right here cause im sick with it cudi they all take the money for granted but dont want to work for it tell me now isnt it funny nah oh its just me myself and i solo ride until i die cause i got me for life  oh i dont need a hand to hold even when the night is cold i got that fire in my soul i dont need anything to get me through the night except the beat thats in my heart yeah its keeping me alive  i dont need anything to make me satisfied  cause the music fills me good and it gets me every time like babababadaba bababadaba  babababadaba bababadaba babababadaba bababadaba cause the music fills me good and it gets me every time yeah lonely nights i laid awake pray the lord my soul to take my hearts become too cold to break know im great but im broke as hell having dreams that im folding cake all my life ive been told to wait but imma get it now yeah its no debate yeah oh its just me myself and i solo ride until i die cause i got me for life  oh i dont need a hand to hold even when the night is cold i got that fire in my soul i dont need anything to get me through the night except the beat thats in my heart yeah its keeping me alive  i dont need anything to make me satisfied  cause the music fills me good and it gets me every time like babababadaba bababadaba babababadaba bababadaba babababadaba bababadaba cause the music fills me good and it gets me every time"". Return only a single word, either POSITIVE, NEGATIVE or NEUTRAL by lyrics: '{texto}'"
    
    #THIRD_COMMAD to FEW-SHOT
    comando = f"The following lyrics is POSITIVE: ""payphone trying call home change spent times gone baby wrong plans made two hard remember people used even harder picture next say late make late try time wasted bridges burned wasted nights turned lights paralyzed still stuck time called love even sun sets paradise payphone trying call home change spent times gone baby wrong plans made two happy ever exist would still holding like fairy tales full one stupid love song sick turned back tomorrow cause forgot yesterday gave love borrow gave away expect fine expect care said bridges burned wasted nights turned lights paralyzed still stuck time called love even sun sets paradise payphone trying call home change spent times gone baby wrong plans made two happy ever exist would still holding like fairy tales full one stupid love song sick payphone baby hang tell need baby begging please go tell need payphone trying call home change spent times gone baby wrong plans made two happy ever exist would still holding like fairy tales full one stupid love song sick payphone"". The following lyrics is NEGATIVE: ""i threw a wish in the well dont ask me ill never tell i looked to you as it fell and now youre in my way i trade my soul for a wish pennies and dimes for a kiss i wasnt looking for this but now youre in my way your stare was holdin ripped jeans skin was showin hot night wind was blowin where you think youre going baby hey i just met you and this is crazy but heres my number so call me maybe its hard to look right at you baby but heres my number so call me maybe hey i just met you and this is crazy but heres my number so call me maybe and all the other boys try to chase me but heres my number so call me maybe you took your time with the call i took no time with the fall you gave me nothing at all but still youre in my way i beg and borrow and steal at first sight and its real i didnt know i would feel it but its in my way your stare was holdin ripped jeans skin was showin hot night wind was blowin where you think youre going baby hey i just met you and this is crazy but heres my number so call me maybe its hard to look right at you baby but heres my number so call me maybe hey i just met you and this is crazy but heres my number so call me maybe and all the other boys try to chase me but heres my number so call me maybe before you came into my life i missed you so bad i missed you so bad i missed you so so bad before you came into my life i missed you so bad and you should know that i missed you so so bad  its hard to look right at you baby but heres my number so call me maybe hey i just met you and this is crazy but heres my number so call me maybe and all the other boys try to chase me but heres my number so call me maybe before you came into my life i missed you so bad i missed you so bad i missed you so so bad before you came into my life i missed you so bad and you should know that so call me maybe"". The following lyrics is NEUTRAL: ""oh its just me myself and i solo ride until i die cause i got me for life oh i dont need a hand to hold even when the night is cold i got that fire in my soul and as far as i can see i just need privacy plus a whole lot of tree fuck all this modesty i just need space to do me get a world that theyre tryna see a stella maxwell right beside of me a ferrari im buyin three a closet of saint laurent get what i want when i want cause this hunger is driving me yeah i just need to be alone i just need to be at home understand what im speaking on if time is money i need a loan but regardless ill always keep keepin on fuck fake friends we dont take ls we just make ms while yall follow we just make trends im right back to work when that break ends oh its just me myself and i solo ride until i die cause i got me for life  oh i dont need a hand to hold even when the night is cold i got that fire in my soul i dont need anything to get me through the night except the beat thats in my heart yeah its keeping me alive  i dont need anything to make me satisfied  cause the music fills me good and it gets me every time yeah and i dont like talking to strangers so get the fuck off me im anxious im tryna be cool but i may just go ape shit say fuck yall to all of yall faces it changes though now that im famous everyone knows how this lifestyle is dangerous but i love it the rush is amazing celebrate nightly and everyone rages i found how to cope with my anger im swimming in money swimming in liquor my liver is muddy but its all good im still sippin this bubbly this shit is lovely this shit aint random i didnt get lucky made it right here cause im sick with it cudi they all take the money for granted but dont want to work for it tell me now isnt it funny nah oh its just me myself and i solo ride until i die cause i got me for life  oh i dont need a hand to hold even when the night is cold i got that fire in my soul i dont need anything to get me through the night except the beat thats in my heart yeah its keeping me alive  i dont need anything to make me satisfied  cause the music fills me good and it gets me every time like babababadaba bababadaba  babababadaba bababadaba babababadaba bababadaba cause the music fills me good and it gets me every time yeah lonely nights i laid awake pray the lord my soul to take my hearts become too cold to break know im great but im broke as hell having dreams that im folding cake all my life ive been told to wait but imma get it now yeah its no debate yeah oh its just me myself and i solo ride until i die cause i got me for life  oh i dont need a hand to hold even when the night is cold i got that fire in my soul i dont need anything to get me through the night except the beat thats in my heart yeah its keeping me alive  i dont need anything to make me satisfied  cause the music fills me good and it gets me every time like babababadaba bababadaba babababadaba bababadaba babababadaba bababadaba cause the music fills me good and it gets me every time"". The following lyrics is POSITIVE: ""yeah its over its over im circling these vultures got me praying man this hunger feeling something rotten last time i saw you said whats up and pushed right through then i tried to catch you but were always on the move and now its over were sober symptoms of the culture and the night aint getting younger last calls around the corner feeling kinda tempted and im pouring out the truth just fading out these talkers cause now all i want is you im saying come over here sit next to me we can see where things go naturally just say the word and ill part the sea just come over here and sit next to me oohoohooh and ill take you high high oohoohooh well i aint frontin my intention got your man outlined in chalk its a midnight intervention got no plans to make it stop last time i saw you said you wanna keep it light but im here to tell you im not trying to change your mind its alright oohoohooh its alright oohoohooh yeah come over here and sit next to me we can see where things go naturally just say the word and ill part the sea just come over here and sit next to me oohoohooh and ill take you high high high high its alright its alright yeah its alright yeah its alright so come over here and sit next to me we can see where things go naturally just say the word and ill part the sea just come over here and sit next to me oohoohooh its alright"". The following lyrics is NEGATIVE: ""wicked wicked wicked wicked wicked tunes you know  wizard  huh wicked wicked wicked wicked wicked hold up wicked wicked wicked wicked wicked woo wicked wicked wicked wicked wicked hold up wicked wicked wicked wicked stand up in the motor bust the dashboard stand up in the motor bust the dashboard hold up wicked wicked wicked wicked wicked wicked wicked wicked wicked wicked candles burnin money burnin graveyard these niggas phantom parkin big dawg barkin hey blood ties on me woah woah woah woah shawty want that wig woah woah woah woah im drippin cartier oh oh oh oh put a gold bird on you thats whats happenin i put that lingo on her she was spanish i fill the one liter up with xannies this continental and its panoramic its complimentary to the savages you fuck around with me it be a tragedy i want green green green no asparagus i drink lean lean lean it aint embarrassin huh wicked wicked wicked wicked wicked hold up wicked wicked wicked wicked wicked woo wicked wicked wicked wicked wicked now she goin know that bitch goin i purchase avianne now she lit huh wedding band rings on me lit huh married to the game im the shit huh woah woah woah woah woah bitch we made men we aint pullin up at cribs that we cant get you cant pull a bitch on instagram i aint hit huh and if she tellin lies about me cause she aint shit uh she want that bigbig dog status i was in the alley with them nickel bags tallied now im taliban gang status thats whats happenin and you niggas cant get close cause you dont know me it aint no more dip and dabbin hangin low key these bitches see me and they panic i cant believe it i was on the corner gamblin huh wicked wicked wicked wicked wicked hold up wicked wicked wicked wicked wicked woo wicked wicked wicked wicked wicked wicked wicked wicked wicked stand up in the motor bust the dashboard stand up in the motor bust the dashboard huh wicked wicked wicked wicked wicked hold up wicked wicked wicked wicked wicked"". Return only a single word, either POSITIVE, NEGATIVE or NEUTRAL by lyrics: '{texto}'"
    
    cabecalho = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {CHAVE_API}"
    }
    
    dados = {
        "prompt": comando,
        #percentagem de acerto
        "temperature": 0,
        #nº máximo caratereres resposta
        "max_tokens": 2047,
        #n.º de respostas
        "n": 1,
        #modo de paragem
        "stop": None,
    }
    
    resposta = requests.post(
        f"https://api.openai.com/v1/engines/{modelo_engine}/completions",
        headers=cabecalho,
        json=dados
    )
    
    resposta_json = resposta.json()
    resultado = resposta_json['choices'][0]['text'].strip()


    data = {
        'resultado': resultado
    }
    
    return data

In [10]:
grupos = dividir_por_tokens(lyrics)
few_shot = []

for grupo in grupos:
    
    resultado = analise_sentimento(grupo)

    few_shot.append(resultado)

KeyError: 'choices'

In [21]:
print(one_shot)

[{'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado': 'NEUTRAL'}, {'resultado

In [11]:
print(grupos)

[['wishin wishin wishin wishin wishin movin calm start trouble tryna keep peaceful struggle pull cuddle like lovin die miss yes see things wishin hope brothers outlive tell story shit different gods plan gods plan hold back sometimes wont feel good sometimes ayy finessed weston road ayy nessed might go god wait go hard southside way make sure north side eat still bad things lot bad things wishin wishin wishin wishin wishin bad things lot bad things wishin wishin wishin wishin wishin ayy ayy say love tell partly love bed momma sorry fifty dub even tatted 81 theyll bring crashers party turn o2 o3 dog without 40 oli thered magine never met broskis gods plan gods plan ayy ayy someone watchin shit close yep close since scarlett road ayy road ayy might go god wait go hard southside ayy way make sure north side eat still bad things lot bad things wishin wishin wishin wishin wishin bad things lot bad things wishin wishin wishin wishin wishin'], ['like walking around old empty house hold hand w

In [17]:
# Abra o arquivo CSV no modo de escrita
with open('few_shot.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Escreva o cabeçalho das colunas
    writer.writerow(['lyrics', 'sentiment'])

    # Escreva os dados
    for i in range(len(grupos)):
        writer.writerow([grupos[i][0], few_shot[i]['resultado']])

print("Dados salvos no arquivo 'zero_shot.csv'")

Dados salvos no arquivo 'zero_shot.csv'
